In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import Regularizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from scipy.ndimage import rotate

# Ensure TensorFlow uses GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.set_memory_growth(tf.config.experimental.list_physical_devices('GPU')[0], True)

# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the input data
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Reshape the input data to add channel dimension
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

# Convert the labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Parameters for Beta distributions
a, b = 2, 4  # You can change these values to (2, 5) or (2, 6) for different shift levels

# Function to rotate images
def rotate_images(images, alpha, beta):
    angles = np.random.beta(alpha, beta, size=images.shape[0]) * 180  # Angles between 0 and 180 degrees
    rotated_images = np.array([rotate(image, angle, axes=(0, 1), reshape=False, mode='nearest') for image, angle in zip(images, angles)])
    return rotated_images

# Rotate images
x_train_rotated = rotate_images(x_train, a, b)
x_test_rotated = rotate_images(x_test, b, a)

# Data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train_rotated)

# Define the Fisher Information computation function
def compute_fisher(model, x, num_samples=30):
    f_accum = []
    for i in range(len(model.weights)):
        f_accum.append(np.zeros(K.int_shape(model.weights[i])))
    f_accum = np.array(f_accum)
    for _ in range(num_samples):
        idx = np.random.randint(0, x.shape[0])
        with tf.GradientTape() as tape:
            model_output = model(x[idx:idx+1])
        grads = tape.gradient(model_output, model.weights)
        for i in range(len(grads)):
            f_accum[i] += tf.square(grads[i])
    f_accum /= num_samples
    return f_accum

# Define the CrlbO regularizer
class CrlbORegularizer(Regularizer):
    def __init__(self, fisher, prior_weights, lambda_=0.1):
        self.fisher = fisher
        self.prior_weights = prior_weights
        self.lambda_ = lambda_

    def __call__(self, x):
        regularization = 0.
        for i in range(len(self.fisher)):
            if self.fisher[i].shape == x.shape:
                regularization += self.lambda_ * tf.reduce_sum(self.fisher[i] * tf.square(x - self.prior_weights[i]))
        return regularization

# Task A: Original Fashion MNIST model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(datagen.flow(x_train_rotated, y_train, batch_size=128), epochs=20, validation_data=(x_test_rotated, y_test))
model.save_weights('FashionMNIST_A.h5')

# Compute Fisher Information for Task A
fisher_info = compute_fisher(model, x_train_rotated)

# Task B: Fashion MNIST with CrlbO regularization
model_CrlbOB = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
model_CrlbOB.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model_CrlbOB.load_weights('FashionMNIST_A.h5')

# Re-apply CrlbO regularizer to the new model
for layer, fisher in zip(model_CrlbOB.layers, fisher_info):
    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer = CrlbORegularizer(fisher, layer.get_weights()[0])

model_CrlbOB.fit(datagen.flow(x_train_rotated, y_train, batch_size=128), epochs=20, validation_data=(x_test_rotated, y_test))

# Evaluate performance
A_original_acc = 100 * model.evaluate(x_test_rotated, y_test)[1]
B_CrlbO_acc = 100 * model_CrlbOB.evaluate(x_test_rotated, y_test)[1]

print("Task A Original Accuracy: %.2f%%" % A_original_acc)
print("Task B CrlbO method Accuracy: %.2f%%" % B_CrlbO_acc)
